In [1]:
# packages to ensure functions run 

import os
from glob import glob
from datetime import datetime

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from itertools import groupby

import geopandas as gpd
import rasterio as rio
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import numpy.ma as ma
from rasterio.plot import plotting_extent
from shapely.geometry import box


%matplotlib inline

# Download data and set working directory
os.chdir(os.path.join(et.io.HOME, 
                      'earth-analytics', 
                      'urban-stormwater-capture'))

In [2]:
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range,
                    pixel_qa_path):
    
    """
    Open, clean, and crop a file into an xarray DataArray.

    Parameters
    -----------
    band_path:string
        A path to the array to be opened
    crop_bound:geopandas GeoDataFrame
        A geopandas dataframe to be used to crop the raster data using rioxarray clip().
    valid_range:tuple (optional)
         A tuple of min and max range of values for the data. Default = None

    Returns
    -----------

    band : xarray DataArray
        End result for a cropped xarray dataarray
    """

#     crop_bound_box = [box(*crop_bound.total_bounds)]

    try:
        band = rxr.open_rasterio(band_path,
                                 masked=True).rio.clip(
                                                        crop_bound.geometry,
                                                        crs=crop_bound.crs,
                                                        all_touched=True,
                                                        from_disk=True).squeeze()
    
        mask = ((band < valid_range[0]) | (band > valid_range[1]))
        band = band.where(~xr.where(mask, True, False))

        # open and mask cloud layer
        cl_mask = (rxr.open_rasterio(pixel_qa_path, masked=True)
                        .rio.clip(
                            crop_bound.geometry,
                            from_disk=True).squeeze())

        # apply cloud mask
        band = band.where(cl_mask.isin(goodQuality)).squeeze()
        
    except:
        raise ValueError(
            "Oops - I couldn't clip your data. This may be due to a crs error.")

    return band

In [3]:

### Follow Sentinel-2 HLS documentation to create masking value references ## 

# Git: https://git.earthdata.nasa.gov/projects/LPDUR/repos/hls-tutorial/browse/HLS_Tutorial.ipynb#processhls

def cloud_mask (fmask):
    
    """
    Create bit dictionary and map Sentinel-2 HLS quality assurance (QA) layers from bits
    

    """

    # bitword dictionaries 

    bitword_order = (1, 1, 1, 1, 1, 1, 2)  # set the number of bits per bitword
    num_bitwords = len(bitword_order)      # Define the number of bitwords based on your input above
    total_bits = sum(bitword_order)        # Should be 8, 16, or 32 depending on datatype

    # Loop to create cloud mask lookup from Git documentation 

    qVals = list(np.unique(fmask))  # Create a list of unique values that need to be converted to binary and decoded

    all_bits = list()
    summary_bits=list()

    goodQuality = []
    for v in qVals:
        all_bits = []
        bits = total_bits
        i = 0

        # Convert to binary based on the values and # of bits defined above:
        bit_val = format(v, 'b').zfill(bits)
#         print('\n' + str(v) + ' = ' + str(bit_val))
        all_bits.append(str(v) + ' = ' + str(bit_val))

        # Go through & split out the values for each bit word based on input above:
        for b in bitword_order:
            prev_bit = bits
            bits = bits - b
            i = i + 1
            if i == 1:
                bitword = bit_val[bits:]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
            elif i == num_bitwords:
                bitword = bit_val[:prev_bit]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
            else:
                bitword = bit_val[bits:prev_bit]
#                 print(' Bit Word ' + str(i) + ': ' + str(bitword)) ## use to print visual tally of bitmap 
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))

#       1 = Cirrus, 2 = Cloud, 3 = Cloud adjacent, 4 = cloud shadow, 5 = snow/ice, 6 = water, 7 = aerosol
        if int(all_bits[1].split(': ')[-1]) < 1 and \
            int(all_bits[2].split(': ')[-1]) < 1 and \
            int(all_bits[3].split(': ')[-1]) < 1 and \
            int(all_bits[4].split(': ')[-1]) < 2 and \
            int(all_bits[5].split(': ')[-1]) < 1 and \
            int(all_bits[6].split(': ')[-1]) < 2 and \
                int(all_bits[7].split(': ')[-1]) < 12:
        
            goodQuality.append(v)
            
    # test values that do not need to be masked 
    return goodQuality

In [4]:
def test_addition (x,
                  y):
    """
    x = first variable
    y = second variable
    
    """
    
    result = x + y
    
    return result